<a href="https://colab.research.google.com/github/ahsank/StockML/blob/main/Backtest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Backtesting
=======================

It uses *backtesting.py* Python framework for [backtesting](https://www.investopedia.com/terms/b/backtesting.asp) trading strategies. See [Quickstart](https://github.com/kernc/backtesting.py/blob/master/doc/examples/Quick%20Start%20User%20Guide.ipynb)


## Data
DataFrame should ideally be indexed with a _datetime index_ (convert it with [`pd.to_datetime()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.to_datetime.html));
otherwise a simple range index will do.

In [303]:
!pip install backtesting

In [304]:
!pip install yahoo_fin

In [306]:
from yahoo_fin import stock_info
tickers = ['ARKK', 'SPY', 'MSFT', 'NVDA', 'PTON', 'DDOG', 'XLE']
num_days = 1200 # 6 years
dfs = {}
for ticker in tickers:
  df = stock_info.get_data(ticker, start_date='2016-01-01')
  df.columns = map(str.title, df.columns)
  df['Unadjusted'] = df.Close
  df.Close = df.Adjclose
  df.drop('Adjclose', axis=1, inplace=True)
  # df = df.tail(num_days)
  dfs[ticker] = df


In [ ]:
# Use Adjclose, Doesn't work
# spy.Close = spy.Adjclose
# spy.drop('Adjclose', axis=1, inplace=True)

In [307]:
import pandas as pd


def SMA(values, n):
    """
    Return simple moving average of `values`, at
    each step taking into account `n` previous values.
    """
    return pd.Series(values).rolling(n).mean()

In [308]:
def ToSeries(values):
  return pd.Series(values)

In [379]:
from backtesting import Strategy
from backtesting.lib import crossover


class SmaCross(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    n1 = 10 # 10
    n2 = 20 # 20

    def init(self):
        # Precompute the two moving averages
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)

    def next(self):
        # If sma1 crosses above sma2, close any existing
        # short trades, and buy the asset
        if crossover(self.sma1, self.sma2):
            self.position.close()
            self.buy()

        # Else, if sma1 crosses below sma2, close any existing
        # long trades, and sell the asset
        elif crossover(self.sma2, self.sma1):
            self.position.close()
            self.sell()

In [ ]:
class AboveSma(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    n1 = 5
    n2 = 200
    n3 = 101 # Should be above 1% SMA

    def init(self):
        # Precompute the two moving averages
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
        self.Close = self.I(ToSeries, self.data.Close)

    def next(self):
        # If price crosses above sma1 and sma2, close any existing
        # short trades, and buy the asset
        if crossover(self.Close, self.sma1) and crossover(self.Close, self.sma2):
            self.position.close()
            self.buy()

        # Else, if price crosses below sma1 and sma2, close any existing
        # long trades
        elif crossover(self.sma1, self.Close) and crossover(self.sma2, self.Close):
            self.position.close()
            # self.sell()

In [ ]:
class CautiousSma(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    n1 = 5
    n2 = 200

    def init(self):
        # Precompute the two moving averages
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
        self.Close = self.I(ToSeries, self.data.Close)

    def next(self):
        # If price crosses above sma1 and sma2, close any existing
        # short trades, and buy the asset
        if crossover(self.Close, self.sma1) and crossover(self.Close, self.sma2):
            self.position.close()
            self.buy()
        # Else, if price crosses below sma1 or sma2, close any existing
        # long trades
        elif crossover(self.sma1, self.Close) or crossover(self.sma2, self.Close):
            self.position.close()
            # self.sell()

In [309]:
def HighRange(values, m, n=0):
    """
    Return High value of range (-m, -n) days
    """
    return pd.Series(values).shift(n).rolling(m).max()

In [310]:
def LowRange(values, m, n=0):
    """
    Return High value of range (-m, -n) days
    """
    return pd.Series(values).shift(n).rolling(m).min()

In [313]:
dfs['SPY']

,Open,High,Low,Close,Volume,Ticker,Unadjusted
2016-01-04,200.490005,201.029999,198.589996,174.043228,222353500,SPY,201.020004
2016-01-05,201.399994,201.899994,200.050003,174.337585,110845800,SPY,201.360001
2016-01-06,198.339996,200.059998,197.600006,172.138428,152112600,SPY,198.820007
2016-01-07,195.330002,197.440002,193.589996,168.008560,213436100,SPY,194.050003
2016-01-08,195.190002,195.850006,191.580002,166.164444,209817200,SPY,191.919998
...,...,...,...,...,...,...,...
2024-04-17,506.049988,506.220001,499.119995,500.549988,75910300,SPY,500.549988
2024-04-18,501.980011,504.130005,498.559998,499.519989,74548100,SPY,499.519989
2024-04-19,499.440002,500.459991,493.859985,495.160004,102129100,SPY,495.160004
2024-04-22,497.829987,502.380005,495.429993,499.720001,67763400,SPY,499.720001


In [419]:
class AboveSmaAndLY(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    n1 = 5
    n2 = 200
    n3 = 200
    n4 = 100
    useyh = False
    daydiff = 20
    lastClose = 0

    def init(self):
        # Precompute the two moving averages
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
        self.Close = self.I(ToSeries, self.data.Close)
        self.YHigh = self.I(HighRange, self.data.Close, self.n3)
        self.LYHigh = self.I(HighRange, self.data.Close, self.n3, self.n3)
        self.YLow = self.I(LowRange, self.data.Close, self.n3)
        self.LYLow = self.I(LowRange, self.data.Close, self.n3, self.n3)

    def next(self):
        close = self.Close[-1]
        closeadj = close*self.n4/100.0
        # If price crosses above sma1 and sma2, close any existing
        # short trades, and buy the asset
        if close > self.sma1[-1] and \
              close > self.sma2[-1] and \
              self.Close.size > self.lastClose + self.daydiff and \
              (close > (self.YLow[-1] + self.LYHigh[-1] + self.YHigh[-1])/3 or not self.useyh):
              # self.position.close()
              # print(self.Close[-1])
              self.buy()

        # Else, if price crosses below sma1 and sma2, close any existing
        # long trades
        elif (self.sma1[-1] > closeadj and \
            self.sma2[-1] > closeadj and \
            ((self.LYHigh[-1] + self.YHigh[-1])/2 > closeadj or not self.useyh)):
            # print(self.position.pl_pct)
            if self.position.is_long:
              self.lastClose = self.Close.size
            self.position.close()
            # self.sell()

In [420]:
bt = Backtest(dfs['NVDA'], AboveSmaAndLY, cash=10_000, commission=0)
stats = bt.run()
stats

Start                     2016-01-04 00:00:00
End                       2024-04-23 00:00:00
Duration                   3032 days 00:00:00
Exposure Time [%]                   57.511962
Equity Final [$]                 174529.00919
Equity Peak [$]                  202474.61179
Return [%]                        1645.290092
Buy & Hold Return [%]            10264.932715
Return (Ann.) [%]                   41.168288
Volatility (Ann.) [%]               49.543363
Sharpe Ratio                         0.830955
Sortino Ratio                        1.894735
Calmar Ratio                         0.927342
Max. Drawdown [%]                  -44.393877
Avg. Drawdown [%]                   -6.057224
Max. Drawdown Duration      535 days 00:00:00
Avg. Drawdown Duration       31 days 00:00:00
# Trades                                    8
Win Rate [%]                             62.5
Best Trade [%]                     374.260395
Worst Trade [%]                     -8.995736
Avg. Trade [%]                    

## Backtesting

 See
[`Backtest`](https://kernc.github.io/backtesting.py/doc/backtesting/backtesting.html#backtesting.backtesting.Backtest)


In [423]:
from backtesting import Backtest

cols = ['Start', 'End', 'Duration', 'Exposure Time [%]', 'Equity Final [$]',
       'Equity Peak [$]', 'Return [%]', 'Buy & Hold Return [%]',
       'Return (Ann.) [%]', 'Volatility (Ann.) [%]', 'Sharpe Ratio',
       'Sortino Ratio', 'Calmar Ratio', 'Max. Drawdown [%]',
       'Avg. Drawdown [%]', 'Max. Drawdown Duration', 'Avg. Drawdown Duration',
       '# Trades', 'Win Rate [%]', 'Best Trade [%]', 'Worst Trade [%]',
       'Avg. Trade [%]', 'Max. Trade Duration', 'Avg. Trade Duration',
       'Profit Factor', 'Expectancy [%]', 'SQN', ]
starr = {}
bts = {}
for ticker in dfs.keys():
  tmpbt = Backtest(dfs[ticker], AboveSmaAndLY, cash=10_000, commission=0)
  tmpstats = bt.run()
  starr[ticker] = tmpstats
  bts[ticker] = tmpbt




In [424]:
statsdf = pd.DataFrame(starr).T[cols]
statsdf.mean()

Start                     2016-01-04 00:00:00
End                       2024-04-23 00:00:00
Duration                   3032 days 00:00:00
Exposure Time [%]                   57.511962
Equity Final [$]                 174529.00919
Equity Peak [$]                  202474.61179
Return [%]                        1645.290092
Buy & Hold Return [%]            10264.932715
Return (Ann.) [%]                   41.168288
Volatility (Ann.) [%]               49.543363
Sharpe Ratio                         0.830955
Sortino Ratio                        1.894735
Calmar Ratio                         0.927342
Max. Drawdown [%]                  -44.393877
Avg. Drawdown [%]                   -6.057224
Max. Drawdown Duration      535 days 00:00:00
Avg. Drawdown Duration       31 days 00:00:00
# Trades                                  8.0
Win Rate [%]                             62.5
Best Trade [%]                     374.260395
Worst Trade [%]                     -8.995736
Avg. Trade [%]                    



[`Backtest.plot()`](https://kernc.github.io/backtesting.py/doc/backtesting/backtesting.html#backtesting.backtesting.Backtest.plot)
method provides the same insights in a more visual form.

In [425]:
bt.plot()

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p4984', ...)

## Optimization

 optimize the two parameters by calling
[`Backtest.optimize()`](https://kernc.github.io/backtesting.py/doc/backtesting/backtesting.html#backtesting.backtesting.Backtest.optimize)


In [395]:
%%time

# stats = bt.optimize(n1=range(5, 30, 5),
#                     n2=range(200, 250, 10),
#                     n3 = range(100, 110, 1),
#                     maximize='Equity Final [$]',
#                     constraint=lambda param: param.n1 < param.n2)
stats = bts['NVDA'].optimize(n1=range(5, 30, 5),
                             n2=range(20, 250, 10),
                              maximize='Equity Final [$]',
                              constraint=lambda param: param.n1 < param.n2)
stats

Backtest.optimize:   0%|          | 0/3 [00:00<?, ?it/s]

CPU times: user 243 ms, sys: 72.9 ms, total: 316 ms
Wall time: 12.8 s


Start                     2016-01-04 00:00:00
End                       2024-04-23 00:00:00
Duration                   3032 days 00:00:00
Exposure Time [%]                   58.660287
Equity Final [$]                281680.998047
Equity Peak [$]                 326813.802246
Return [%]                         2716.80998
Buy & Hold Return [%]            10264.932715
Return (Ann.) [%]                   49.555815
Volatility (Ann.) [%]               52.634487
Sharpe Ratio                         0.941508
Sortino Ratio                        2.343073
Calmar Ratio                         1.287992
Max. Drawdown [%]                  -38.475247
Avg. Drawdown [%]                   -5.492179
Max. Drawdown Duration      518 days 00:00:00
Avg. Drawdown Duration       28 days 00:00:00
# Trades                                   41
Win Rate [%]                        56.097561
Best Trade [%]                     347.270823
Worst Trade [%]                     -8.995736
Avg. Trade [%]                    

Check`stats['_strategy']`

In [396]:
stats._strategy

<Strategy AboveSmaAndLY(n1=5,n2=190)>

In [397]:
bts['NVDA'].plot(plot_volume=False, plot_pl=False)

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p4472', ...)

Strategy optimization managed to up its initial performance _on in-sample data_ by almost 50% and even beat simple
[buy & hold](https://en.wikipedia.org/wiki/Buy_and_hold).
In real life optimization, however, do **take steps to avoid
[overfitting](https://en.wikipedia.org/wiki/Overfitting)**.

## Trade data

In addition to backtest statistics returned by
[`Backtest.run()`](https://kernc.github.io/backtesting.py/doc/backtesting/backtesting.html#backtesting.backtesting.Backtest.run)
shown above, you can look into _individual trade returns_ and the changing _equity curve_ and _drawdown_ by inspecting the last few, internal keys in the result series.

In [398]:
stats.tail()

Expectancy [%]                                            14.702499
SQN                                                        1.231193
_strategy                                AboveSmaAndLY(n1=5,n2=190)
_equity_curve                        Equity  DrawdownPct Drawdow...
_trades               Size  EntryBar  ExitBar  EntryPrice   Exit...
dtype: object

The columns should be self-explanatory.

In [399]:
starr['NVDA']['_equity_curve']  # Contains equity/drawdown curves. DrawdownDuration is only defined at ends of DD periods.

,Equity,DrawdownPct,DrawdownDuration
2016-01-04,10000.000000,0.000000,NaT
2016-01-05,10000.000000,0.000000,NaT
2016-01-06,10000.000000,0.000000,NaT
2016-01-07,10000.000000,0.000000,NaT
2016-01-08,10000.000000,0.000000,NaT
...,...,...,...
2024-04-17,252202.130066,0.115400,NaT
2024-04-18,254110.143982,0.108708,NaT
2024-04-19,228697.137390,0.197844,NaT
2024-04-22,238651.135193,0.162931,NaT


In [422]:
stats['_trades'].tail(50)  # Contains individual trade data

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,237,401,699,42.097500,60.542500,4371.464928,0.438150,2017-08-07,2018-10-11,430 days
1,332,893,901,43.224998,40.297501,-971.929291,-0.067727,2019-07-23,2019-08-02,10 days
2,325,922,1057,41.145000,50.227501,2951.812649,0.220744,2019-09-03,2020-03-17,196 days
3,227,1078,1303,71.769997,121.302498,11243.877777,0.690156,2020-04-16,2021-03-09,327 days
4,193,1324,1529,142.527496,220.119995,14975.352264,0.544404,2021-04-08,2022-01-28,295 days
5,175,1550,1554,242.910004,228.169998,-2579.500961,-0.060681,2022-03-01,2022-03-07,6 days
6,215,1749,1752,185.309998,168.639999,-3584.049606,-0.089957,2022-12-13,2022-12-16,3 days
7,213,1773,2089,170.360001,807.950012,135806.672470,3.742604,2023-01-19,2024-04-23,460 days


Learn more by exploring further
[examples](https://kernc.github.io/backtesting.py/doc/backtesting/index.html#tutorials)
or find more framework options in the
[full API reference](https://kernc.github.io/backtesting.py/doc/backtesting/index.html#header-submodules).